In [14]:
# import mamba_ssm
import networks.mamba_sys as mamba_sys
import torch
import torch as tc
import my_mambanet
import importlib
import torch.nn as nn

importlib.reload(my_mambanet)

<module 'my_mambanet' from '/home/collettida/myprojects/ExploringMamba/code/my_mambanet.py'>

In [16]:
loss = torch.nn.MSELoss()
# model = mamba_sys.().to('cuda')
model = my_mambanet.MambaUNet(patch_size=3,in_chans=30, num_classes=2).to('cuda')
inp = torch.randn(16,30,21,21).cuda()
# int = torch.randn(16,1,21,21).cuda()
out = model(inp)
l = loss(out, torch.randn_like(out))
l.backward()
print(out.shape)
print("done")



---final upsample expand_first---
Input shape to MambaUNet: torch.Size([16, 30, 24, 24])
After Patch Embedding: torch.Size([16, 8, 8, 96])
After layer: torch.Size([16, 4, 4, 192])
After layer: torch.Size([16, 2, 2, 384])
After layer: torch.Size([16, 1, 1, 768])
After layer: torch.Size([16, 1, 1, 768])
After norm: torch.Size([16, 1, 1, 768])
Decoder layer index: 0 Input shape: torch.Size([16, 1, 1, 768])
After decoder layer: torch.Size([16, 2, 2, 384])
Decoder layer index: 1 Input shape: torch.Size([16, 2, 2, 384])
After decoder layer: torch.Size([16, 4, 4, 192])
Decoder layer index: 2 Input shape: torch.Size([16, 4, 4, 192])
After decoder layer: torch.Size([16, 8, 8, 96])
Decoder layer index: 3 Input shape: torch.Size([16, 8, 8, 96])
After decoder layer: torch.Size([16, 8, 8, 96])
After norm up: torch.Size([16, 8, 8, 96])
Before final upsample: torch.Size([16, 8, 8, 96])
After final upsample: torch.Size([16, 2, 24, 24])
torch.Size([16, 2, 21, 21])
done


In [3]:
class PatchEmbedVideo(nn.Module):
    """ Video to Patch Embedding
    Args:
        img_size (int): Frame size.  Default: 21
        in_chans (int): Number of input image channels. Default: 3
        n_frames (int): Number of frames. Default: 8
        patch_size (int): Patch token size. Default: 3
        stride (int): Stride of the patch embedding. Default: None
        embed_dim (int): Number of linear projection output channels. Default: 96.
        norm_layer (nn.Module, optional): Normalization layer. Default: None
    """
    def __init__(self, image_size=21, n_frames=8, patch_size=3, 
                 stride=None, in_chans=2, embed_dim=96, groups=1, 
                 norm_layer=None, **kwargs):
        super().__init__()
        if isinstance(patch_size, int):
            patch_size = (patch_size, patch_size)
        if stride is None:
            stride = patch_size
        if isinstance(stride, int):
            stride = (stride, stride)

        self.proj = nn.Conv2d(in_chans, 
                              embed_dim, 
                              kernel_size=patch_size,
                              stride=patch_size,
                              groups=groups)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None 
        
    def forward(self, x: tc.Tensor):
        """ Forward function.
        Args:
            x: (tc.Tensor) Input video of shape (B, T, C, H, W)
        Returns:
            tc.Tensor: Patch embedded video of shape (B, H', W', embed_dim)
        """
        B, T, C, H, W = x.shape
         
        for t in range(T):
            x_t = x[:, t, :, :, :]  # (B, C, H, W)
            x_t = self.proj(x_t)  # (B, embed_dim, H', W')
            if t == 0:
                print(x_t.shape)
                x_out = x_t.unsqueeze(1)  # (B, 1, embed_dim, H', W')
                print(x_out.shape)
            else:
                x_out = tc.cat((x_out, x_t.unsqueeze(1)), dim=1)  # (B, T, embed_dim, H', W')
        

        if self.norm is not None:
            x_out = self.norm(x_out)
        
        return x_out



In [4]:
pev = PatchEmbedVideo(image_size=21, n_frames=15, patch_size=3, in_chans=2, embed_dim=96).to('cuda')

inp = torch.randn(16,15,2,21,21).cuda()
out = pev(inp)
print(out.shape)


torch.Size([16, 96, 7, 7])
torch.Size([16, 1, 96, 7, 7])
torch.Size([16, 15, 96, 7, 7])


In [25]:
import math
from einops import rearrange, repeat 
import torch.nn.functional as F
from mamba_ssm.ops.selective_scan_interface import selective_scan_fn

class SS2D(nn.Module):
    r""" Mamba SSM layer for 2D inputs.
    2D-Selective-Scan for Vision Data (SS2D)
    """
    def __init__(
        self,
        d_model,
        d_state=16,
        d_conv=3,
        expand=2,
        dt_rank="auto",
        dt_min=0.001,
        dt_max=0.1,
        dt_init="random",
        dt_scale=1.0,
        dt_init_floor=1e-4,
        dropout=0.0,
        conv_bias=True,
        bias=False,
        device=None,
        dtype=None,
        **kwargs,
    ):
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_conv = d_conv 
        self.expand = expand
        self.d_inner = int(self.d_model * self.expand)
        self.dt_rank = math.ceil(self.d_model / 16) if dt_rank == "auto" else dt_rank

        # Input projection to xz
        self.in_proj = nn.Linear(self.d_model,
                                 self.d_inner*2,
                                 bias=bias,
                                 **factory_kwargs)
        # DW conv
        self.conv2d = nn.Conv2d(
            in_channels=self.d_inner,
            out_channels=self.d_inner,
            groups=self.d_inner,
            bias=conv_bias,
            kernel_size=d_conv,
            padding=(d_conv-1) // 2,
            **factory_kwargs,
        )
        self.act = nn.SiLU()

        # 4 projections for [dt, B, C] calculation
        self.x_proj = (
            nn.Linear(self.d_inner, (self.dt_rank+self.d_state*2), bias=False, **factory_kwargs),
            nn.Linear(self.d_inner, (self.dt_rank+self.d_state*2), bias=False, **factory_kwargs),
            nn.Linear(self.d_inner, (self.dt_rank+self.d_state*2), bias=False, **factory_kwargs),
            nn.Linear(self.d_inner, (self.dt_rank+self.d_state*2), bias=False, **factory_kwargs),
        )
        self.x_proj_weight = nn.Parameter(tc.stack([t.weight for t in self.x_proj], dim=0)) # (K=4, N, inner)
        del self.x_proj

        
        self.dt_projs = (
            self.dt_init(self.dt_rank, self.d_inner, dt_scale, dt_init, dt_min, dt_max, dt_init_floor, **factory_kwargs),
            self.dt_init(self.dt_rank, self.d_inner, dt_scale, dt_init, dt_min, dt_max, dt_init_floor, **factory_kwargs),
            self.dt_init(self.dt_rank, self.d_inner, dt_scale, dt_init, dt_min, dt_max, dt_init_floor, **factory_kwargs),
            self.dt_init(self.dt_rank, self.d_inner, dt_scale, dt_init, dt_min, dt_max, dt_init_floor, **factory_kwargs),
        )
        self.dt_projs_weight = nn.Parameter(tc.stack([t.weight for t in self.dt_projs], dim=0)) # (K=4, inner, rank)
        self.dt_projs_bias = nn.Parameter(tc.stack([t.bias for t in self.dt_projs], dim=0)) # (K=4, inner)
        del self.dt_projs

        self.A_logs = self.A_log_init(self.d_state, self.d_inner, copies=4, merge=True) # (K=4, D, N)
        self.Ds = self.D_init(self.d_inner, copies=4, merge=True) # (K=4, D, N)

        self.forward_core = self.forward_corev0
        
        self.out_norm = nn.LayerNorm(self.d_inner)
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=bias, **factory_kwargs)
        self.dropout = nn.Dropout(dropout) if dropout > 0. else None

    @staticmethod
    def dt_init(dt_rank, d_inner, dt_scale=1.0, dt_init="random", 
                dt_min=0.001, dt_max=0.1, dt_init_floor=1e-4, **factory_kwargs):
        dt_proj = nn.Linear(dt_rank, d_inner, bias=True, **factory_kwargs)

        # Initialize special dt projection to preserve variance at initialization
        dt_init_std = dt_rank**-0.5 * dt_scale
        if dt_init == "constant":
            nn.init.constant_(dt_proj.weight, dt_init_std)
        elif dt_init == "random":
            nn.init.uniform_(dt_proj.weight, -dt_init_std, dt_init_std)
        else:
            raise NotImplementedError
        
        # Initialize dt bias so that F.softplus(dt_bias) is between dt_min and dt_max
        dt = tc.exp(
            tc.rand(d_inner, **factory_kwargs) * (math.log(dt_max) - math.log(dt_min))
            + math.log(dt_min)).clamp(min=dt_init_floor)
        # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        inv_dt = dt + tc.log(-tc.expm1(-dt))
        with tc.no_grad():
            dt_proj.bias.copy_(inv_dt)
        # Our initialization would set all Linear.bias to zero, need to mark this one as _no_reinit
        dt_proj.bias._no_reinit = True
        
        return dt_proj

    @staticmethod
    def A_log_init(d_state, d_inner, copies=1, device=None, merge=True):
        # S4D real initialization
        A = repeat(
            tc.arange(1, d_state + 1, dtype=tc.float32, device=device),
            "n -> d n",
            d=d_inner,
        ).contiguous()
        A_log = tc.log(A)  # Keep A_log in fp32
        if copies > 1:
            A_log = repeat(A_log, "d n -> r d n", r=copies)
            if merge:
                A_log = A_log.flatten(0, 1)
        A_log = nn.Parameter(A_log)
        A_log._no_weight_decay = True
        return A_log

    @staticmethod
    def D_init(d_inner, copies=1, device=None, merge=True):
        # D "skip" parameter
        D = tc.ones(d_inner, device=device)
        if copies > 1:
            D = repeat(D, "n1 -> r n1", r=copies)
            if merge:
                D = D.flatten(0, 1)
        D = nn.Parameter(D)  # Keep in fp32
        D._no_weight_decay = True
        return D

    def forward_corev0(self, x: tc.Tensor):
        r""" Core forward function for SS2D.
        Performs SS on 4 sequence obtained by inspecting the input 2D feature map
        along different directions: (H→W), (W→H), (H→W flipped), (W→H flipped).
        """
        self.selective_scan = selective_scan_fn

        B, C, H, W = x.shape
        L = H*W # sequence length = N of tokens
        K = 4  # number of projections

        x_hwwh = tc.stack([x.view(B, -1, L), tc.transpose(x, dim0=2, dim1=3).contiguous().view(B, -1, L)], dim=1).view(B, 2, -1, L)
        xs = tc.cat([x_hwwh, tc.flip(x_hwwh, dims=[-1])], dim=1) # (b, k, d, l)

        x_bdl = tc.einsum("b k d l, k c d -> b k c l", xs.view(B, K, -1, L), self.x_proj_weight)
        dts, Bs, Cs = tc.split(x_bdl, [self.dt_rank, self.d_state, self.d_state], dim=2)  
        dts = tc.einsum("b k r l, k d r -> b k d l", dts.view(B,K,-1,L), self.dt_projs_weight)

        xs = xs.view(B, -1, L)
        dts = dts.contiguous().float().view(B, -1, L)
        Bs = Bs.float().view(B, K, -1, L) # (b, k, d_state, l)
        Cs = Cs.float().view(B, K, -1, L) # (b, k, d_state, l)
        
        Ds = self.Ds.float().view(-1)
        As = -tc.exp(self.A_logs.float()).view(-1, self.d_state) 
        dt_projs_bias = self.dt_projs_bias.float().view(-1)

        out_y = self.selective_scan(
            xs, dts,
            As, Bs, Cs, Ds, z=None,
            delta_bias=dt_projs_bias,
            delta_softplus=True,
            return_last_state=False,
        ).view(B, K, -1, L)
        assert out_y.dtype == tc.float

        inv_y = tc.flip(out_y[:, 2:4], dims=[-1]).view(B, 2, -1, L)
        wh_y =  tc.transpose(out_y[:, 1].view(B, -1, H, W), dim0=2, dim1=3).contiguous().view(B, -1, L)
        invwh_y = tc.transpose(inv_y[:, 1].view(B, -1, H, W), dim0=2, dim1=3).contiguous().view(B, -1, L)        
        y = out_y[:, 0] + inv_y[:, 0] + wh_y + invwh_y
        y = tc.transpose(y, dim0=1, dim1=2).contiguous().view(B, H, W, -1)
        y = self.out_norm(y).to(x.dtype)

        return y
    
    def forward(self, x: tc.Tensor, **kwargs):
        B, H, W, C = x.shape
        print(f"x input shape: {x.shape}")
        xz = self.in_proj(x)
        print(f"xz shape: {xz.shape}")
        x, z = xz.chunk(2, dim=-1) # each: (B, H, W, d_inner)
        print(f"x shape after chunk: {x.shape}")
        print(f"z shape after chunk: {z.shape}")
        
        x = x.permute(0, 3, 1, 2).contiguous()
        x = self.conv2d(x)
        x = self.act(x)
        y = self.forward_core(x)
        y = y * F.silu(z)
        out = self.out_proj(y)
        if self.dropout is not None:
            out = self.dropout(out)
        return out

In [26]:
ss2d = SS2D(d_model=96, d_state=16, d_conv=3).to('cuda')
inp = torch.randn(16,21,21,96).cuda()
out = ss2d(inp)
print(out.shape)

x input shape: torch.Size([16, 21, 21, 96])
xz shape: torch.Size([16, 21, 21, 384])
x shape after chunk: torch.Size([16, 21, 21, 192])
z shape after chunk: torch.Size([16, 21, 21, 192])
torch.Size([16, 21, 21, 96])
